# Preamble

In [1]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not
defined type <=>

__P1:__ $\forall x. \mathrm{Knight}(x) \vee \mathrm{Knave}(x)$, where $\mathrm{Knave}(x) \equiv \neg \mathrm{Knight}(x)$

__P2:__ $\forall x. \mathrm{Knight}(x) \rightarrow \forall p. (\mathrm{Says}(x, p) \rightarrow p)$

__P3:__ $\forall x. \mathrm{Knave}(x) \rightarrow \forall p. (\mathrm{Says}(x, p) \rightarrow \neg p)$

__Puzzle:__ $\{\mathrm{P1},\;\mathrm{P2},\;\mathrm{P3}\} \vdash \forall x. \neg \mathrm{Says}(x, \mathrm{Knave}(x))$


In [2]:
trait Inhabitant:
    type Knight
    type Knave = Not[Knight]
    type Says[_]

defined trait Inhabitant

In [3]:
trait KnightsAndKnaves:
    // ∀ x. Inhabitant(x) 🠂 Knight(x) ∨ Knave(x)
    val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // ∀ p. ∀ x. Knight(x) 🠂 Says(x, p) 🠂 p 
    val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P

    // ∀ p. ∀ x. Knight(x) 🠂 Says(x, p) 🠂 p 
    val P3: [P] => (x: Inhabitant) => x.Knave => x.Says[P] => Not[P]

defined trait KnightsAndKnaves

In [4]:
def puzzle1(KK: KnightsAndKnaves): (x: Inhabitant) => Not[x.Says[x.Knave]] = 
    (x: Inhabitant) => (s: x.Says[x.Knave]) => 
        KK.P1(x) match
            case Left(xIsKnight: x.Knight) => 
                KK.P2[x.Knave](x)(xIsKnight)(s)(xIsKnight) : Nothing
            case Right(xIsKnave) => 
                KK.P3[x.Knave](x)(xIsKnave)(s)(xIsKnave) : Nothing

defined function puzzle1

In [5]:
object Sugar:

    implicit class NoOp[P](np: Not[P]):
        def contradicts(p: P): Nothing = np(p)

    def eitherKnightOrKnave[A](x: Inhabitant): KnightsAndKnaves ?=> (x.Knight ?=> A, x.Knave ?=> A) => A = 
        P ?=> (f, g) => P.P1(x).fold(a => f(using a), a => g(using a))

    def knightsAreTruthful[A](x: Inhabitant): KnightsAndKnaves ?=> x.Knight ?=> x.Says[A] => A = 
        P ?=> xIsKnight ?=> xSaysA => P.P2(x)(xIsKnight)(xSaysA)

    def knavesAreLiers[A](x: Inhabitant): KnightsAndKnaves ?=> x.Knave ?=> x.Says[A] => Not[A] = 
        P ?=> xIsKnave ?=> xSaysA => P.P3(x)(xIsKnave)(xSaysA)

defined object Sugar

In [6]:
import Sugar._

import Sugar._


### Puzzle 1

You have met a group of 3 islanders. Their names are Bob, Wallace, and Joan.

* Joan says: Wallace is a knight.
* Wallace says: Bob never lies.
* Joan says: Bob is my type.

Answer: 

Bob, Wallace and Joan are all knights.

Reasoning:

* A knight or a knave will say they are the same type as a knight. So when Joan says they are the same type as Bob, we know that Bob is a knight.
* All islanders will call one of their same kind a knight. So when Wallace says that Bob is a knight, we know that Bob and Wallace are the same type. Since Bob is a knight, then Wallace is a knight.
* All islanders will call one of their same kind a knight. So when Joan says that Wallace is a knight, we know that Wallace and Joan are the same type. Since Wallace is a knight, then Joan is a knight.

In [7]:
def deMorgan[P, Q](n: Not[Either[P, Q]]): (Not[P], Not[Q]) = 
    (p => n(Left(p)), q => n(Right(q)))

defined function deMorgan

In [8]:
def proof1(using KnightsAndKnaves)(x: Inhabitant, y: Inhabitant): 
        x.Says[Either[(x.Knight, y.Knight), (x.Knave, y.Knave)]] => y.Knight = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            knightsAreTruthful(x)(xSays) match
                case Left(xIsKnight, yIsKnight) => yIsKnight
                case Right(xIsKnave, _) => xIsKnave(xIsKnight),
        xIsKnave ?=> 
            val l: (Not[(x.Knight, y.Knight)], Not[(x.Knave, y.Knave)]) = deMorgan(knavesAreLiers(x)(xSays))
            eitherKnightOrKnave(y)(
                yIsKnight ?=> yIsKnight , 
                yIsKnave ?=> l._2(xIsKnave, yIsKnave)))

defined function proof1

In [9]:
def proof2(using KnightsAndKnaves)(x: Inhabitant, y: Inhabitant): 
        x.Says[y.Knight] => Either[(x.Knight, y.Knight), (x.Knave, y.Knave)] = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            Left(xIsKnight, knightsAreTruthful(x)(xSays)),
        xIsKnave ?=> 
            Right(xIsKnave, knavesAreLiers(x)(xSays)))

defined function proof2

In [10]:
def proof3[P, Q]: (Either[(P, Q), (Not[P], Not[Q])], Q) => P = 
    case (Left(p, _),_) => p
    case (Right(_, nq),q) => nq(q)

defined function proof3

In [11]:
def puzzle(using KnightsAndKnaves)(
    joan: Inhabitant, wallace: Inhabitant, bob: Inhabitant): 
    (joan.Says[wallace.Knight], 
     wallace.Says[bob.Knight], 
     joan.Says[Either[(joan.Knight, bob.Knight),(joan.Knave, bob.Knave)]]) => (joan.Knight, wallace.Knight, bob.Knight) = 
    (joanSays1, wallaceSays, joanSays2) => 
        // A knight or a knave will say they are the same type as a knight. 
        // So when Joan says they are the same type as Bob, we know that Bob is a knight.
        val bobIsKnight: bob.Knight = proof1(joan, bob)(joanSays2)
        // All islanders will call one of their same kind a knight. So when Wallace says that Bob is a knight, we know that 
        // Bob and Wallace are the same type. 
        val sameTypeWB: Either[(wallace.Knight, bob.Knight), (wallace.Knave, bob.Knave)] = proof2(wallace, bob)(wallaceSays)
        // Since Bob is a knight, then Wallace is a knight.
        val wallaceIsKnight: wallace.Knight = proof3(sameTypeWB, bobIsKnight)
        // All islanders will call one of their same kind a knight. So when Joan says that Wallace is a knight, 
        // we know that Wallace and Joan are the same type.
        val sameTypeJW: Either[(joan.Knight, wallace.Knight), (joan.Knave, wallace.Knave)] = proof2(joan, wallace)(joanSays1)
        //  Since Wallace is a knight, then Joan is a knight.
        val joanIsKnight: joan.Knight = proof3(sameTypeJW, wallaceIsKnight)
        
        (joanIsKnight, wallaceIsKnight, bobIsKnight)
    


defined function puzzle

---

### Puzzle 2

You have met a group of 4 islanders. Their names are Quentin, Beatrix, Robert, and Nancy.

* Robert says: Beatrix is a knave.
* Robert says: Nancy is lying.
* Quentin says: Robert is a knave and I am a knave.

Answer: 

The knaves were Quentin, Nancy, and Beatrix, and the only knight was Robert.

Reasoning:

* Because Quentin said 'Robert is a knave and I am a knave,' we know Quentin is not making a true statement. (If it was true, the speaker would be a knight claiming to be a knave, which cannot happen.) Therefore, Quentin is a knave and Robert is a knight.
* All islanders will call a member of the opposite type a knave. So when Robert says that Beatrix is a knave, we know that Beatrix and Robert are opposite types. Since Robert is a knight, then Beatrix is a knave.
* All islanders will call a member of the opposite type a knave. So when Robert says that Nancy is a knave, we know that Nancy and Robert are opposite types. Since Robert is a knight, then Nancy is a knave.

In [12]:
def deMorgan2[P, Q](thirdMiddleP: Either[P, Not[P]])(n: Not[(P, Q)]): Either[Not[P], Not[Q]] = 
    thirdMiddleP.fold(
            p => Right((q) => n((p, q)))
            ,
            np => Left((p) => np(p))
        )

trait DN:
    def left[P]: Not[Not[P]] => P
    def right[P]: P => Not[Not[P]] =
        p => np => np(p)

defined function deMorgan2
defined trait DN

In [13]:
def commutativeEither[A,B] : Either[A, B] => Either[B, A] =
    case Left(a) => Right(a)
    case Right(b) => Left(b)

trait DN:
    def left[P]: Not[Not[P]] => P
    def right[P]: P => Not[Not[P]] =
        p => np => np(p)

defined function commutativeEither
defined trait DN

In [14]:
def proof1_1(using KK : KnightsAndKnaves)(x: Inhabitant, y: Inhabitant):
        x.Says[(x.Knave, y.Knave)] => x.Knave =
        xSays => eitherKnightOrKnave(x)(
                xIsKnight ?=>
                        knightsAreTruthful(x)(xSays)._1(xIsKnight),
                xIsKnave ?=>
                        xIsKnave
        )

defined function proof1_1

In [15]:
def proof1_2(using KK : KnightsAndKnaves)(using dn : DN)(x: Inhabitant, y: Inhabitant):
        x.Says[(x.Knave, y.Knave)] => x.Knave => (x.Knave, y.Knight) =
        xSays => xIsKnave => eitherKnightOrKnave(x)(
                xIsKnight ?=>
                        xIsKnave(xIsKnight),
                xIsKnave ?=>
                        val lie : Not[(x.Knave, y.Knave)] = knavesAreLiers(x)(xSays)
                        val knaveORnnknave : Either[x.Knave, Not[x.Knave]] = commutativeEither(KK.P1(x)).fold(v => Left(v), g => Right(dn.right(g)))
                        val eitherLie: Either[Not[x.Knave], Not[y.Knave]] = deMorgan2(knaveORnnknave)(lie)
                        eitherLie.fold(
                                xNotKnave => xNotKnave(xIsKnave),
                                yNotKnave => (xIsKnave, dn.left(yNotKnave))
                        )
        )

defined function proof1_2

In [16]:
def proof2(using KnightsAndKnaves)(using dn: DN)(x: Inhabitant, y: Inhabitant): 
    x.Says[y.Knave] => Either[(x.Knight, y.Knave), (x.Knave, y.Knight)] = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            Left((xIsKnight, knightsAreTruthful(x)(xSays))),
        xIsKnave ?=> 
            Right((xIsKnave, dn.left(knavesAreLiers(x)(xSays))))
    )

defined function proof2

In [17]:
def proof3[P, Q]: (Either[(P, Not[Q]), (Not[P], Q)], P) => Not[Q] = 
    case (Left(_, nq),_) => nq
    case (Right(np, _),p) => np(p)

defined function proof3

In [18]:
def puzzle2(using KnightsAndKnaves)(
    quentin: Inhabitant, beatrix: Inhabitant, robert: Inhabitant, nancy: Inhabitant)(using dn: DN):
    (robert.Says[beatrix.Knave], 
     robert.Says[nancy.Knave], 
     quentin.Says[(quentin.Knave, robert.Knave)]) => (quentin.Knave, beatrix.Knave, robert.Knight, nancy.Knave) = 
    (robertSays1, robertSays2, quentinSays) => 
        // Because Quentin said 'Robert is a knave and I am a knave,' we know Quentin is not making a true statement. 
        // (If it was true, the speaker would be a knight claiming to be a knave, which cannot happen.)
        val quentinIsKnave : quentin.Knave = proof1_1(quentin, robert)(quentinSays)
        // Therefore, Quentin is a knave and Robert is a knight. 
        val quentinIsKnaverobertIsKnight: (quentin.Knave, robert.Knight) = proof1_2(quentin, robert)(quentinSays)(quentinIsKnave)
        val robertIsKnight : robert.Knight = quentinIsKnaverobertIsKnight._2
        // All islanders will call a member of the opposite type a knave. 
        // So when Robert says that Beatrix is a knave, we know that Beatrix and Robert are opposite types. 
        val oppositeTypeRB: Either[(robert.Knight, beatrix.Knave), (robert.Knave, beatrix.Knight)] = proof2(robert, beatrix)(robertSays1)
        // Since Robert is a knight, then Beatrix is a knave.
        val beatrixIsKnave: beatrix.Knave = proof3(oppositeTypeRB, robertIsKnight)
        // All islanders will call a member of the opposite type a knave. 
        // So when Robert says that Nancy is a knave, we know that Nancy and Robert are opposite types. 
        val oppositeTypeRN: Either[(robert.Knight, nancy.Knave), (robert.Knave, nancy.Knight)] = proof2(robert, nancy)(robertSays2)
        // Since Robert is a knight, then Nancy is a knave.
        val nancyIsKnave: nancy.Knave = proof3(oppositeTypeRN, robertIsKnight)
        
        (quentinIsKnave, beatrixIsKnave, robertIsKnight, nancyIsKnave)
    


defined function puzzle2

---

### Puzzle 3

You have met a group of 6 islanders. Their names are Bob, Neil, Francine, Zelda, Henry, and Wallace.   

* Bob says: Francine always tells the truth.
* Neil says: Bob never tells the truth.
* Francine says: Neil is untruthful.
* Francine says: Bob is my type.
* Henry says: Wallace never lies.
* Henry says: Zelda always lies.
* Francine says: Wallace is not my type.

Answer: 

The knaves were Neil, Henry, and Wallace, and the knights were Bob, Francine, and Zelda.

Reasoning:

* A knight or a knave will say they are the same type as a knight. So when Francine says they are the same type as Bob, we know that Bob is a knight.
* Both knights and knaves will say they are not the same type as a knave. So when Francine says they are a different type than Wallace, we know that Wallace is a knave.
* All islanders will call one of their same kind a knight. So when Bob says that Francine is a knight, we know that Francine and Bob are the same type. Since Bob is a knight, then Francine is a knight.
* All islanders will call a member of the opposite type a knave. So when Neil says that Bob is a knave, we know that Bob and Neil are opposite types. Since Bob is a knight, then Neil is a knave.
* All islanders will call one of their same kind a knight. So when Henry says that Wallace is a knight, we know that Wallace and Henry are the same type. Since Wallace is a knave, then Henry is a knave.
* All islanders will call a member of the opposite type a knave. So when Henry says that Zelda is a knave, we know that Zelda and Henry are opposite types. Since Henry is a knave, then Zelda is a knight.

In [19]:
def proof1(using KnightsAndKnaves)(x: Inhabitant, y: Inhabitant): 
        x.Says[Either[(x.Knight, y.Knight), (x.Knave, y.Knave)]] => y.Knight = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            knightsAreTruthful(x)(xSays) match
                case Left(xIsKnight, yIsKnight) => yIsKnight
                case Right(xIsKnave, _) => xIsKnave(xIsKnight),
        xIsKnave ?=> 
            val l: (Not[(x.Knight, y.Knight)], Not[(x.Knave, y.Knave)]) = deMorgan(knavesAreLiers(x)(xSays))
            eitherKnightOrKnave(y)(
                yIsKnight ?=> yIsKnight , 
                yIsKnave ?=> l._2(xIsKnave, yIsKnave)))

defined function proof1

In [20]:
def proof2(using KnightsAndKnaves)(x: Inhabitant, y: Inhabitant): 
        x.Says[Either[(x.Knight, y.Knave), (x.Knave, y.Knight)]] => y.Knave = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            knightsAreTruthful(x)(xSays) match
                case Left(xIsKnight, yIsKnave) => yIsKnave
                case Right(xIsKnave, _) => xIsKnave(xIsKnight),
        xIsKnave ?=> 
            val l: (Not[(x.Knight, y.Knave)], Not[(x.Knave, y.Knight)]) = deMorgan(knavesAreLiers(x)(xSays))
            eitherKnightOrKnave(y)(
                yIsKnight ?=> l._2(xIsKnave, yIsKnight), 
                yIsKnave ?=> yIsKnave)
        )

defined function proof2

In [21]:
def proof3(using KnightsAndKnaves)(x: Inhabitant, y: Inhabitant): 
        x.Says[y.Knight] => Either[(x.Knight, y.Knight), (x.Knave, y.Knave)] = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            Left(xIsKnight, knightsAreTruthful(x)(xSays)),
        xIsKnave ?=> 
            Right(xIsKnave, knavesAreLiers(x)(xSays)))

defined function proof3

In [22]:
def proof4[P, Q]: (Either[(P, Q), (Not[P], Not[Q])], P) => Q = 
    case (Left(_, q),_) => q
    case (Right(np, _),p) => np(p)

defined function proof4

In [23]:
def proof5(using KnightsAndKnaves)(using dn: DN)(x: Inhabitant, y: Inhabitant): 
    x.Says[y.Knave] => Either[(x.Knight, y.Knave), (x.Knave, y.Knight)] = 
    xSays => eitherKnightOrKnave(x)(
        xIsKnight ?=> 
            Left((xIsKnight, knightsAreTruthful(x)(xSays))),
        xIsKnave ?=> 
            Right((xIsKnave, dn.left(knavesAreLiers(x)(xSays))))
    )

defined function proof5

In [24]:
def proof6[P, Q]: (Either[(P, Not[Q]), (Not[P], Q)], Q) => Not[P] = 
    case (Left(_, nq),q) => nq(q)
    case (Right(np, _),_) => np

defined function proof6

In [25]:
def proof7[P, Q]: (Either[(P, Q), (Not[P], Not[Q])], Not[Q]) => Not[P] = 
    case (Left(_, q),nq) => nq(q)
    case (Right(np,_),_) => np

defined function proof7

In [26]:
def proof8[P, Q]: (Either[(P, Not[Q]), (Not[P], Q)], Not[P]) => Q = 
    case (Left(p, _),np) => np(p)
    case (Right(_, q),_) => q

defined function proof8

In [27]:
def puzzle3(using KnightsAndKnaves)(
    bob: Inhabitant, neil: Inhabitant, francine: Inhabitant, zelda: Inhabitant, henry: Inhabitant, wallace: Inhabitant)(using dn: DN):
        (bob.Says[francine.Knight], 
         neil.Says[bob.Knave], 
         francine.Says[neil.Knave],
         francine.Says[Either[(francine.Knight, bob.Knight),(francine.Knave, bob.Knave)]],
         henry.Says[wallace.Knight],
         henry.Says[zelda.Knave],
         francine.Says[Either[(francine.Knight, wallace.Knave),(francine.Knave, wallace.Knight)]]) => (neil.Knave, henry.Knave, wallace.Knave, bob.Knight, francine.Knight, zelda.Knight) = 
        (bobSays, neilSays, francineSays1, francineSays2, henrySays1, henrySays2, francineSays3) => 
                // A knight or a knave will say they are the same type as a knight. 
                // So when Francine says they are the same type as Bob, we know that Bob is a knight.
                val bobIsKnight: bob.Knight = proof1(francine, bob)(francineSays2)
                // Both knights and knaves will say they are not the same type as a knave. 
                // So when Francine says they are a different type than Wallace, we know that Wallace is a knave.
                val wallaceIsKnave: wallace.Knave = proof2(francine, wallace)(francineSays3)
                // All islanders will call one of their same kind a knight. 
                // So when Bob says that Francine is a knight, we know that Francine and Bob are the same type. 
                val sameTypeBF: Either[(bob.Knight, francine.Knight),(bob.Knave, francine.Knave)] = proof3(bob, francine)(bobSays)
                // Since Bob is a knight, then Francine is a knight.
                val francineIsKnight: francine.Knight = proof4(sameTypeBF, bobIsKnight)
                // All islanders will call a member of the opposite type a knave. 
                // So when Neil says that Bob is a knave, we know that Bob and Neil are opposite types. 
                val oppositeTypeNB: Either[(neil.Knight, bob.Knave), (neil.Knave, bob.Knight)] = proof5(neil, bob)(neilSays)
                // Since Bob is a knight, then Neil is a knave. 
                val neilIsKnave: neil.Knave = proof6(oppositeTypeNB, bobIsKnight)
                // All islanders will call one of their same kind a knight. 
                // So when Henry says that Wallace is a knight, we know that Wallace and Henry are the same type. 
                val sameTypeHW: Either[(henry.Knight, wallace.Knight),(henry.Knave, wallace.Knave)] = proof3(henry, wallace)(henrySays1)
                // Since Wallace is a knave, then Henry is a knave.
                val henryIsKnave: henry.Knave = proof7(sameTypeHW, wallaceIsKnave)
                // All islanders will call a member of the opposite type a knave. 
                // So when Henry says that Zelda is a knave, we know that Zelda and Henry are opposite types. 
                val oppositeTypeHZ: Either[(henry.Knight, zelda.Knave), (henry.Knave, zelda.Knight)] = proof5(henry, zelda)(henrySays2)
                // Since Henry is a knave, then Zelda is a knight.
                val zeldaIsKnight: zelda.Knight = proof8(oppositeTypeHZ, henryIsKnave)
                (neilIsKnave, henryIsKnave, wallaceIsKnave, bobIsKnight, francineIsKnight, zeldaIsKnight)

defined function puzzle3